# Setup/Imports

In [188]:
# Getting the post reader
from post_parser_record import PostParserRecord
post_reader = PostParserRecord("Posts_Coffee.xml")

# Getting the tokenizer
!pip install nltk 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords  

stop_words = set(stopwords.words('english'))

import string
import re
import math

# Cleans the text

def clean_string(s):
  # Removes HTML tags
  CLEANR = re.compile('<.*?>') 
  s = re.sub(CLEANR, '', s)
  # Removes newlines and adds a space so that words do not combine
  s = s.replace('\n', ' ') 
  # Removes punctuation
  s = s.translate(str.maketrans('','',string.punctuation))
  
  return s

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Creating Index for the three models

### TF-IDF & BM25

In [189]:
# Creates the Inverted Index
inverted_index = {}
tf_index = {}
tf_ind = {}
tf_idf_index= {}
c = len(post_reader.map_questions) + len(post_reader.map_just_answers) # Total doc count
avg = 0
doc_length = {}
# Gathering Question Data
for question_id in post_reader.map_questions:
  doc_freq = {}

  # Gets question
  question = post_reader.map_questions[question_id]

  # Combines question body and title
  question_b = question.body.lower().strip()
  question_t = question.title.lower().strip()
  question_bt = question_b + " " + question_t

  # Cleaning
  question_bt = clean_string(question_bt)

  # Tokenizing
  token_words = nltk.word_tokenize(question_bt)
  token_words = [w for w in token_words if not w.lower() in stop_words]
  l = len(token_words)
  
  avg += l # To store avg length of all docs
  doc_length.update({question_id: l})  # Used for BM25

  for token in token_words:   # Loop through tokens
    if token in inverted_index:     # Token is already in index
      doc_freq = inverted_index[token]  # Sets doc_freq to the current postings/counts for the token
      if question_id in doc_freq:
        freq = doc_freq[question_id] + 1
        doc_freq.update({question_id: freq})
        inverted_index.update({token: doc_freq})
      else:
        doc_freq.update({question_id: 1})
    else:   # Token is not in index, adds to index with a frequency of 1
      doc_freq = {question_id: 1}    
      inverted_index.update({token: doc_freq})

      # Calculates TF 
  for token in token_words: 
    if token in tf_index:
      tf_ind = tf_index[token]
      tf = (inverted_index[token][question_id])/l
      tf_ind.update({question_id: tf})
      tf_index.update({token: tf_ind})
    else:
      tf_ind = {}
      tf = (inverted_index[token][question_id])/l
      tf_ind.update({question_id: tf})
      tf_index.update({token: tf_ind})

# Gathering Answer Data
for answer_id in post_reader.map_just_answers:
  doc_freq = {}

  # Gets answer and text
  answer = post_reader.map_just_answers[answer_id]
  answer_text = answer.body.lower().strip()

  # Cleaning
  answer_text = clean_string(answer_text)

  # Tokenizing
  token_words2 = nltk.word_tokenize(answer_text)
  token_words2 = [w for w in token_words2 if not w.lower() in stop_words]
  l = len(token_words2)
  
  avg += l # To store avg length of all docs
  doc_length.update({answer_id: l})  # Used for BM25

  for token in token_words2:   # Loop through tokens
    if token in inverted_index:     # Token is already in index
      doc_freq = inverted_index[token]  
      if answer_id in doc_freq:
        freq = doc_freq[answer_id] + 1
        doc_freq.update({answer_id: freq})
        inverted_index.update({token: doc_freq})
      else:
        doc_freq.update({answer_id: 1})
    else:   # Token is not in index, adds to index with a frequency of 1
      doc_freq = {answer_id: 1}     
      inverted_index.update({token: doc_freq})

     # Calculates TF 
  for token in token_words2: 
    if token in tf_index:
      tf_ind = tf_index[token]
      tf = (inverted_index[token][answer_id])/l
      tf_ind.update({answer_id: tf})
      tf_index.update({token: tf_ind})
    else:
      tf_ind = {}
      tf = (inverted_index[token][answer_id])/l
      tf_ind.update({answer_id: tf})
      tf_index.update({token: tf_ind})

# Calculates the average words
avg = avg/c
# Sorting keys 
sorted_keys = sorted(inverted_index.keys())
inverted_index = {key:inverted_index[key] for key in sorted_keys}

# Sorting values
for word in inverted_index:
  inverted_index[word] = dict(sorted(inverted_index[word].items(), key=lambda item: item[1], reverse=True))

# Calculates TF-IDF
for word in inverted_index:
  tf_idf = {}
  for id in tf_index[word]:
    IDF = math.log(c /len(inverted_index[word]))
    tf_idf_score = IDF * tf_index[word][id]
    tf_idf.update({id: tf_idf_score})
    tf_idf_index.update({word : tf_idf})

# Sorting values of tf_idf_index
for word in tf_idf_index:
  tf_idf_index[word] = dict(sorted(tf_idf_index[word].items(), key=lambda item: item[1], reverse=True))

### VSM

In [190]:
lst_terms = []
stop_words = set(stopwords.words('english'))

# First making temp dict for making vectors
temp_dic = {}
count = 0

# Looping through Questions for text and input into temp dic
for question_id in post_reader.map_questions:
  question = post_reader.map_questions[question_id]
  text = question.body.strip() + " " + question.title.strip()
  text = text.lower()
  text = clean_string(text)

  filtered_words = nltk.word_tokenize(text)
  filtered_words = [w for w in filtered_words if not w.lower() in stop_words]

  lst_terms.extend(filtered_words)

for term in set(lst_terms):
  if term not in temp_dic:
    temp_dic[term] = count
    count += 1

# Looping through Answers for text and input into temp dic
for ans_id in post_reader.map_just_answers:
  answer = post_reader.map_just_answers[ans_id]
  text = answer.body.strip()
  text = text.lower()
  text = clean_string(text)

  filtered_words = nltk.word_tokenize(text)
  filtered_words = [w for w in filtered_words if not w.lower() in stop_words]

  lst_terms.extend(filtered_words)

for term in set(lst_terms):
  if term not in temp_dic:
    temp_dic[term] = count
    count += 1

# dict of question id/answer id as the key and doc vector as value
dic_doc_vector = {}

# Loop for questions
for question_id in post_reader.map_questions:
  # Cleaning the text up and tokenizing
  question = post_reader.map_questions[question_id]
  doc_vector = [0] * len(temp_dic.keys())
  text = question.body.strip() + " " + question.title.strip()
  text = text.lower()
  text = clean_string(text)

  filtered_words = nltk.word_tokenize(text)
  filtered_words = [w for w in filtered_words if not w.lower() in stop_words]

  # Loop through tokens setting the index of the doc vector to the corresponding tfidf value for the term
  for token in filtered_words:
    if token in temp_dic:
      index = temp_dic[token]
      doc_vector[index] = tf_idf_index[token][question_id]
  dic_doc_vector[question_id] = doc_vector

# Loop for answers
for ans_id in post_reader.map_just_answers:
  # Cleaning the text up and tokenizing
  answer = post_reader.map_just_answers[ans_id]
  doc_vector = [0] * len(temp_dic.keys())
  text = answer.body
  text = text.lower()
  text = clean_string(text)

  filtered_words = nltk.word_tokenize(text)
  filtered_words = [w for w in filtered_words if not w.lower() in stop_words]

  # Loop through tokens setting the index of the doc vector to the corresponding tfidf value for the term
  for token in filtered_words:
    if token in temp_dic:
      index = temp_dic[token]
      doc_vector[index] = tf_idf_index[token][ans_id]
  dic_doc_vector[ans_id] = doc_vector

# Implementation of 3 retrieval models


### Method to print top k ranked results

In [191]:
# Method to print the first k search results by rank. For this project I set k to 50
def top_k(results, k):

  if len(results)==0:
    print("No results")
    return
  top_k = results[:k]
  rank = 1
  print("Rank\tPost ID")
  for item in top_k:
    print(str(rank) + "\t" + str(item))
    rank+=1

### TF-IDF

In [192]:
def retrieve_by_tfidf(dict1, dict2):
  # These are doc lists in sorted order
  list1 = list(sorted(dict1))
  list2 = list(sorted(dict2))
  doc_count = {}
  matches = []
  i = 0
  j = 0
  count = 0

  # Loop to run through each doc for first term
  while i < len(list1) and j < len(list2):
      if list1[i] < list2[j]:
        count = dict1[list1[i]]
        doc_count.update({list1[i]: count})
        matches.append(list1[i])
        i += 1
      elif list2[j] < list1[i]:
        count = dict2[list2[j]]
        doc_count.update({list2[j]: count})
        j += 1
      else:
        count = dict2[list2[j]] + dict1[list1[i]]
        doc_count.update({list2[j]: count})
        i += 1
        j += 1
  
  # Add rest of list1 to matches
  while i < len(list1):
      count = dict1[list1[i]]
      doc_count.update({list1[i]: count})
      i += 1
  
  # Add rest of list2 to matches
  while j < len(list2):
      count = dict2[list2[j]]
      doc_count.update({list2[j]: count})
      j += 1

  # Sorts by tf-idf
  doc_count = dict(sorted(doc_count.items(), key=lambda item: item[1], reverse=True))

  return doc_count

In [193]:
# Function to do retrieval, ranking by frequency of the words in the document
def tfidf_search (query):
  # Breaks up search terms
  query = clean_string(query)
  search_terms = query.lower().split()
  terms = []
  
  # Case for all inputs not in index
  for term in search_terms:
    if term in tf_idf_index:
      break
    elif term not in tf_idf_index and term != search_terms[len(search_terms) - 1]:
      search_terms.remove(term)
    elif term == search_terms[len(search_terms) - 1]:
      return []
  
  # Case for looking at inputs
  for term in search_terms:
    if term in tf_idf_index:
      terms.append(term)

  # If search terms is empty returns no results
  if len(terms) == 0:
    return []

  # Single search
  if len(terms) == 1:
    return list(tf_idf_index[terms[0]].keys())
  
  else: # Search for more than 1 term
    matches = []
    temp_dic = retrieve_by_tfidf(tf_idf_index[terms[0]], tf_idf_index[terms[1]])
    terms.remove(terms[0])
    terms.remove(terms[0])
    
    if len(terms) == 0:
      return list(temp_dic.keys())
    else:
      for term in terms:
        temp_dic = retrieve_by_tfidf(temp_dic, tf_idf_index[term])

      # Sorting and converting to list for output
      temp_dic = dict(sorted(temp_dic.items(), key=lambda item: item[1], reverse=True))
      matches = list(temp_dic.keys())
      return matches

### VSM

In [194]:
# Method to speed up computing the cosine simularity calculation

from numba import jit
import numpy as np
@jit(nopython=True)
def cosine_similarity_numba(u:np.ndarray, v:np.ndarray):
    assert(u.shape[0] == v.shape[0])
    uv = 0
    uu = 0
    vv = 0
    for i in range(u.shape[0]):
        uv += u[i]*v[i]
        uu += u[i]*u[i]
        vv += v[i]*v[i]
    cos_theta = 1
    if uu!=0 and vv!=0:
        cos_theta = uv/np.sqrt(uu*vv)
    return cos_theta

In [195]:
def vsm (query):
  search_terms = [] # List of search terms
  term_freq = {}  # Dictionary holding the term and frequency of the query terms
  length = len(query.split()) # Length of the query
  results = {} # Resulting dictionary holding doc and cosign simularity

  # Breaks up search terms
  query = clean_string(query)
  for term in query.split():
    if term in lst_terms:
      if term in search_terms:
        freq = term_freq[term] + 1
        term_freq.update({term: freq})
      else:
        term_freq.update({term : 1})
      search_terms.append(term)

  # Creates empty query vector
  query_vector = [0] * len(temp_dic.keys())

  # Loop through terms making query vector index equal to corresponding tfidf value
  for term in search_terms:
    if term in temp_dic:
      IDF = math.log(c /len(inverted_index[term]))
      tf_idf_score = IDF * (term_freq[term] / length)
      index = temp_dic[term]
      query_vector[index] = tf_idf_score
  
  # Caculates Cosign simularity between query vector and each foc
  b = query_vector
  for doc in dic_doc_vector:
    a = dic_doc_vector[doc]
    cos_sim = cosine_similarity_numba(np.array(a),np.array(b))
    results.update({doc : cos_sim})
  
  # Sorting
  results = dict(sorted(results.items(), key=lambda item: item[1], reverse=True))
  matches = list(results.keys())
  return matches

### BM25

In [196]:
def bm25 (query):
  search_terms = []

  # Breaks up search terms
  query = clean_string(query)
  for term in set(query.split()):
    if term in lst_terms:
      search_terms.append(term)
  
  temp_dict = {}
  query_dict = {}
  result_dict = {}
  doc_set = set()

  # Calculation and making a dict
  for term in search_terms:
    for doc in tf_idf_index[term]:
      doc_set.add(doc)
      if doc in temp_dict:
        score = temp_dict[doc] + calc_bm25(term, doc)
        temp_dict.update({doc : score})
      else:
        score = calc_bm25(term, doc)
        temp_dict.update({doc : score})
    query_dict.update({term : temp_dict})

  # Single search
  if len(search_terms) == 1:
    dic = query_dict[search_terms[0]]
    dic = dict(sorted(dic.items(), key=lambda item: item[1], reverse=True))
    return list(dic.keys())

  # Multi search
  # Gets the score for each term in each doc and then adds the score and stores in a dictionary mapping of doc to overall query score
  for term in search_terms:
    for doc in doc_set:
      if doc in query_dict[term]:
        if doc in result_dict:
          score = result_dict[doc] + query_dict[term][doc]
          result_dict.update({doc : score})
        else:
          result_dict.update({doc : score})

  result_dict = dict(sorted(result_dict.items(), key=lambda item: item[1], reverse=True))
  matches = list(result_dict.keys())
  return matches

In [197]:
# Helper method to calculate and return the value needed for BM25 model
def calc_bm25 (term, doc):
  # Hyperparameters
  k = 1.2
  b = 0.75
  # Calculation
  idf = tf_idf_index[term][doc]
  partial_score = ((k + 1) * inverted_index[term][doc]) / (k * ((1 - b) + b * (doc_length[doc]/avg)) + inverted_index[term][doc])
  score = idf * partial_score
  return score

# Testing of each model for given queries

#### a. espresso

In [198]:
print("TF-IDF Search")
top_k(tfidf_search("espresso"), 5)
print("VSM Search")
top_k(vsm("espresso"), 5)
print("BM25 Search")
top_k(bm25("espresso"), 5)

TF-IDF Search
Rank	Post ID
1	4404
2	2867
3	3168
4	3904
5	3800
VSM Search
Rank	Post ID
1	2116
2	5528
3	470
4	466
5	2095
BM25 Search
Rank	Post ID
1	4404
2	2867
3	3168
4	4258
5	3800


#### b. turkish coffee

In [199]:
print("TF-IDF Search")
top_k(tfidf_search("turkish coffee"), 5)
print("VSM Search")
top_k(vsm("turkish coffee"), 5)
print("BM25 Search")
top_k(bm25("turkish coffee"), 5)

TF-IDF Search
Rank	Post ID
1	5094
2	5182
3	483
4	209
5	3074
VSM Search
Rank	Post ID
1	5094
2	209
3	2394
4	3074
5	483
BM25 Search
Rank	Post ID
1	5094
2	5182
3	483
4	209
5	3074


#### c. making a decaffeinated coffee

In [200]:
print("TF-IDF Search")
top_k(tfidf_search("making a decaffeinated coffee"), 5)
print("VSM Search")
top_k(vsm("making a decaffeinated coffee"), 5)
print("BM25 Search")
top_k(bm25("making a decaffeinated coffee"), 5)

TF-IDF Search
Rank	Post ID
1	204
2	120
3	3293
4	2897
5	373
VSM Search
Rank	Post ID
1	120
2	204
3	2897
4	2555
5	373
BM25 Search
Rank	Post ID
1	204
2	120
3	3293
4	2897
5	373


#### d. can I use the same coffee grounds twice?


In [201]:
print("TF-IDF Search")
top_k(tfidf_search("can I use the same coffee grounds twice?"), 5)
print("VSM Search")
top_k(vsm("can I use the same coffee grounds twice?"), 5)
print("BM25 Search")
top_k(bm25("can I use the same coffee grounds twice?"), 5)

TF-IDF Search
Rank	Post ID
1	2683
2	1749
3	3258
4	3966
5	183
VSM Search
Rank	Post ID
1	2683
2	1749
3	3258
4	3966
5	4087
BM25 Search
Rank	Post ID
1	2683
2	3258
3	1749
4	183
5	2585


### Table

In [202]:
a1 = tfidf_search("espresso")
b1 = vsm("espresso")
c1 = bm25("espresso")
a2 = tfidf_search("turkish coffee")
b2 = vsm("turkish coffee")
c2 = bm25("turkish coffee")
a3 = tfidf_search("making a decaffeinated coffee")
b3 = vsm("making a decaffeinated coffee")
c3 = bm25("making a decaffeinated coffee")
a4 = tfidf_search("can I use the same coffee grounds twice?")
b4 = vsm("can I use the same coffee grounds twice?")
c4 = bm25("can I use the same coffee grounds twice?")

In [203]:
from prettytable import PrettyTable

myTable = PrettyTable(["Query", "TF-IDF", "VSM", "BM25"])

# Adds rows
myTable.add_row(["espresso", a1[:5], b1[:5], c1[:5]])
myTable.add_row(["turkish coffee", a2[:5], b2[:5], c2[:5]])
myTable.add_row(["making a decaffeinated coffee", a3[:5], b3[:5], c3[:5]])
myTable.add_row(["can I use the same coffee grounds twice?", a4[:5], b4[:5], c4[:5]])

print(myTable)

+------------------------------------------+--------------------------------+--------------------------------+--------------------------------+
|                  Query                   |             TF-IDF             |              VSM               |              BM25              |
+------------------------------------------+--------------------------------+--------------------------------+--------------------------------+
|                 espresso                 | [4404, 2867, 3168, 3904, 3800] |  [2116, 5528, 470, 466, 2095]  | [4404, 2867, 3168, 4258, 3800] |
|              turkish coffee              |  [5094, 5182, 483, 209, 3074]  |  [5094, 209, 2394, 3074, 483]  |  [5094, 5182, 483, 209, 3074]  |
|      making a decaffeinated coffee       |  [204, 120, 3293, 2897, 373]   |  [120, 204, 2897, 2555, 373]   |  [204, 120, 3293, 2897, 373]   |
| can I use the same coffee grounds twice? | [2683, 1749, 3258, 3966, 183]  | [2683, 1749, 3258, 3966, 4087] | [2683, 3258, 1749, 183, 2